In [ ]:
import os
import time
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

In [ ]:
client= OpenAI(api_key= os.getenv("OPENAI_API_KEY"))


In [ ]:
# Test upload a file
import requests
from io import BytesIO

client = OpenAI()

def create_file(client, file_path):
    
    with open(file_path, "rb") as file_content:
        result = client.files.create(
            file=file_content,
            purpose="assistants"
        )
    print(result.id)
    return result.id

file_id = create_file(client, 'Scraping Data/About Us.md')


file-GGomcjXZ6P2j72tinqsBUh


In [8]:
# Create vector store
vector_store = client.vector_stores.create(
    name="anytime_fitness_poc"
)
print(vector_store.id)

vs_684ab7f2a0d88191b1c23da94ddb322a


In [10]:
# Add file to vector store
add_file_result= client.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file_id
)
print(add_file_result)

VectorStoreFile(id='file-GGomcjXZ6P2j72tinqsBUh', created_at=1749727262, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_684ab7f2a0d88191b1c23da94ddb322a', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))


In [16]:
# File Search Tool:
response = client.responses.create(
    model="gpt-4o-mini",
    input="What is anytime fitness about? The output should be a few lines",
    tools=[{
        "type": "file_search",
        "vector_store_ids": ["vs_684ab7f2a0d88191b1c23da94ddb322a"]
    }]
)
print(response)

Response(id='resp_684ab9d74fec81a2aedecfdabc4566ee08603258e9a3a306', created_at=1749727703.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseFileSearchToolCall(id='fs_684ab9d7e4ec81a2b9a01f241db53f9208603258e9a3a306', queries=['What is Anytime Fitness about?'], status='completed', type='file_search_call', results=None), ResponseOutputMessage(id='msg_684ab9d94fec81a2aa3284ac09f102ff08603258e9a3a306', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-GGomcjXZ6P2j72tinqsBUh', index=455, type='file_citation', filename='About Us.md')], text='Anytime Fitness is a global leader in 24-hour fitness, offering state-of-the-art equipment and certified trainers in a supportive community. The organization emphasizes enriching lives through health, fitness, and convenience, aiming to facilitate individual fitness journeys while also focusing on franchise growth. With clubs in 48 countr

In [ ]:
# --- Cleanup Cell: Delete Test Vector Store and File ---

vector_store_id_to_delete = "vs_684ab7f2a0d88191b1c23da94ddb322a"
file_id_to_delete = "file-GGomcjXZ6P2j72tinqsBUh"

print(f"Attempting to delete Vector Store: {vector_store_id_to_delete}")
try:
    # 1. Delete the Vector Store
    # Deleting the store automatically handles the disassociation of any files within it.
    response_vs = client.vector_stores.delete(
      vector_store_id=vector_store_id_to_delete
    )
    print(f"Successfully deleted vector store: {response_vs.id}, status: {response_vs.deleted}")
except Exception as e:
    # This will catch errors if the store was already deleted or doesn't exist.
    print(f"Error deleting vector store: {e}")

print("-" * 20)

print(f"Attempting to delete File: {file_id_to_delete}")
try:
    # 2. Delete the File
    # This removes the file completely from your OpenAI organization's storage.
    response_file = client.files.delete(
      file_id=file_id_to_delete
    )
    print(f"Successfully deleted file: {response_file.id}, status: {response_file.deleted}")
except Exception as e:
    # This will catch errors if the file was already deleted or doesn't exist.
    print(f"Error deleting file: {e}")